In [11]:
import pandas as pd
import re
import os

In [12]:
def log_read(log_file):
    print(f'read {log_file}.')
    # 读取日志文件
    log_data = pd.read_csv(log_file, sep=r'\]|\[', header=None,
                           usecols=[1, 3, 5, 6],
                           names=['log_time', 'address', 'log_type', 'commit'])
    log_data.log_time = pd.to_datetime(
        log_data.log_time, format='%Y-%m-%d %H:%M:%S')
    return log_data

In [13]:
def get_synoses(commit):
    # 日志报错内容缩写分类
    synoses = commit
    if re.search('Stream closed', commit):
        synoses = 'Stream closed'
    elif re.search('Restart', commit):
        synoses = 'Restart'
    elif re.search('more than 200ms', commit):
        synoses = 'more than 200ms'
    elif re.search('Receive disconnect message', commit):
        synoses = 'Receive disconnect'
    elif re.search('from read get image result is null', commit):
        synoses = 'image result is null'
    elif re.search('Login device failed error', commit):
        synoses = 'Login failed'
    elif re.search('Get video from sdk failed', commit):
        synoses = 'Get video failed'
    elif re.search('convert  yuv data to ipl  failed', commit):
        synoses = 'convert yuv data failed'
    elif re.search('slow', commit):
        synoses = 'slow'
    return synoses

In [14]:
def get_log_data(log_fold):
    log_data = pd.DataFrame()
    for file in os.listdir(log_fold):
        if file[:9] != 'mylogfile':
            continue
        file = log_fold + '/' + file
        log_data = log_data.append(log_read(file))
    log_data.drop_duplicates(inplace=True)
    log_data['synopses'] = log_data.commit.apply(get_synoses)
    return log_data

In [15]:
if __name__ == '__main__':
    log_data = get_log_data('/Users/panbingqing/Documents/工作/城市大脑/长沙调控/log_file')
    print('drop_duplicates.')
    log_data.drop_duplicates(inplace=True)

    # 根据address匹配对应路口道路
    print('match address.')
    camera_info = pd.read_csv('/Users/panbingqing/Documents/工作/城市大脑/长沙调控/data_inline/camera_info',
                              usecols=['摄像头编号', '摄像头IP地址', '摄像头名称'],
                              index_col='摄像头IP地址')
    camera_info = camera_info.to_dict(orient='index')

    log_data['camera_id'] = log_data.address.apply(
        lambda x: camera_info[x]['摄像头编号'])
    log_data['road_name'] = log_data.address.apply(
        lambda x: camera_info[x]['摄像头名称'])

    log_data.to_csv('/Users/panbingqing/Documents/工作/城市大脑/长沙调控/data/log_data.csv', index=False)

read /Users/panbingqing/Documents/工作/城市大脑/长沙调控/log_file/mylogfile 6.


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


read /Users/panbingqing/Documents/工作/城市大脑/长沙调控/log_file/mylogfile 7.
read /Users/panbingqing/Documents/工作/城市大脑/长沙调控/log_file/mylogfile.
read /Users/panbingqing/Documents/工作/城市大脑/长沙调控/log_file/mylogfile 5.
read /Users/panbingqing/Documents/工作/城市大脑/长沙调控/log_file/mylogfile 2.
read /Users/panbingqing/Documents/工作/城市大脑/长沙调控/log_file/mylogfile 3.
read /Users/panbingqing/Documents/工作/城市大脑/长沙调控/log_file/mylogfile 4.
drop_duplicates.
match address.
